# 크롤링

### 크롬 드라이버 다운 필요
(크롬이 최신 버전이라는 전제하에 시작, 작업 디렉토리까지의 절대경로에 한글이 있으면 오류가 발생할 수 있음)
1. 크롬 접속 후 우측 상단 점 3개 클릭 → 도움말 → Chrome 정보 클릭
2. 자신의 Chrome 버전 확인
3. 링크 접속 https://googlechromelabs.github.io/chrome-for-testing/
4. 자신과 똑같거나 거의 비슷한 버전의 왼쪽 단어 클릭(Stable, Beta, Dev, Canary 중 하나)
5. chromedriver 항목중에서 자신의 OS 환경에 맞는 URL 주소 복사
6. 인터넷 창에 URL 복사 붙여넣기하면 다운받을 수 있음
7. 자신의 작업 디렉토리에 압축 해제

In [2]:
# 터미널에 붙여넣기
# pip install bs4 selenium

In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
import datetime
import os
import getpass
import time

In [6]:
# ★★★★★★★★★★★★★★★★★★★★★★★★
# < 직접 수정하는 부분 >
# ★★★★★★★★★★★★★★★★★★★★★★★★

show_browser = False  # 크롬 브라우저 창 표시 여부, 이것을 True로 설정하면 실시간으로 수집되는 과정을 볼 수 있습니다.

now = datetime.datetime.now().strftime('%Y-%m-%d_%H%M%S')

# 원하는 검색어 설정
search_keyword = "머신러닝" # 딥러닝, 머신러닝

# 작업 디렉토리까지의 절대경로 입력 + 마지막에 / 입력
load_folder = 'C:/Users/kimsu/Desktop/crawling/'

# 데이터를 저장할 파일 이름 설정
crawling_data_file_name = f'riss_crawling_{search_keyword}_{now}.csv'

# 직접 검색하여 총 몇 페이지인지 확인 후 입력
page_count = 3

In [7]:
URL_TEMPLATE = (
    f"https://www.riss.kr/search/Search.do?"
    f"isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery={search_keyword}"
    f"&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3ASCOPUS%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88"
    f"regnm%3AKCI%EB%93%B1%EC%9E%AC%ED%9B%84%EB%B3%B4%E2%97%88regnm%3ASCIE%E2%97%88regnm%3AESCI%E2%97%88regnm%3ASCI%E2%97%88"
    f"regnm%3A%EA%B5%AC%29KCI%EB%93%B1%EC%9E%AC%28%ED%86%B5%ED%95%A9%29%E2%97%88regnm%3ASSCI%E2%97%88regnm%3A3903%E2%97%88"
    f"regnm%3A%EB%93%B1%EC%9E%AC%ED%9B%84%EB%B3%B4%E2%97%88regnm%3A02%E2%97%88pyear%3A2024%E2%97%88pyear%3A2023%E2%97%88pyear%3A2022"
    f"%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016"
    f"%E2%97%88pyear%3A2015%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1"
    f"%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BSCOPUS%5D%40%40regnm%3ASCOPUS%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4"
    f"+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95"
    f"%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%ED%9B%84%EB%B3%B4%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%ED%9B%84%EB%B3%B4%E2%97%88%EB%93%B1%EC%9E%AC"
    f"%EC%A0%95%EB%B3%B4+%5BSCIE%5D%40%40regnm%3ASCIE%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BESCI%5D%40%40regnm%3AESCI%E2%97%88%EB%93%B1"
    f"%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BSCI%5D%40%40regnm%3ASCI%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5B%EA%B5%AC%29KCI%EB%93%B1%EC%9E%AC%28"
    f"%ED%86%B5%ED%95%A9%29%5D%40%40regnm%3A%EA%B5%AC%29KCI%EB%93%B1%EC%9E%AC%28%ED%86%B5%ED%95%A9%29%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4"
    f"+%5BSSCI%5D%40%40regnm%3ASSCI%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5B3903%5D%40%40regnm%3A3903%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95"
    f"%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%ED%9B%84%EB%B3%B4%5D%40%40regnm%3A%EB%93%B1%EC%9E%AC%ED%9B%84%EB%B3%B4%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95"
    f"%EB%B3%B4+%5B02%5D%40%40regnm%3A02%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2024%5D%40%40pyear%3A2024%E2%97%88%EB%B0%9C%ED%96%89"
    f"%EC%97%B0%EB%8F%84+%5B2023%5D%40%40pyear%3A2023%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2022%5D%40%40pyear%3A2022%E2%97%88%EB%B0%9C"
    f"%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88"
    f"%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018"
    f"%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear"
    f"%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2="
    f"&iStartCount={{iStartCount}}"
    f"&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn="
    f"&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword={search_keyword}"
    f"&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10"
    f"&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query={search_keyword}"
)

In [8]:
# 여러 페이지를 돌면서 링크를 가져오는 함수
def get_paper_links(driver, url):
    driver.get(url)
    time.sleep(1.5)  # 페이지 로딩을 기다립니다.

    soup = bs(driver.page_source, 'html.parser')
    links = []

    for a in soup.select("#divContent > div > div.rightContent.wd756 > div > div.srchResultW > div.srchResultListW > ul > li > div.cont.ml60 > p.title > a"):
        links.append('https://www.riss.kr' + a.get('href', ''))

    return links

# 논문 세부 정보를 가져오는 함수
def get_paper_details(driver, url):
    driver.get(url)
    time.sleep(1.5)  # 페이지 로딩을 기다립니다.

    soup = bs(driver.page_source, 'html.parser')

    title = soup.select_one('h3.title')
    title_kor = title.get_text(strip=True) if title else ""
    
    authors = soup.select_one('#thesisInfoDiv > div.infoDetail.on > div.infoDetailL > ul > li:nth-of-type(1) > div > p')
    authors_text = authors.get_text(strip=True) if authors else ""
    
    keywords = soup.select_one('#thesisInfoDiv > div.infoDetail.on > div.infoDetailL > ul > li:nth-of-type(7) > div > p')
    keywords_text = keywords.get_text(strip=True) if keywords else ""

    korean_abstract = soup.select_one('#additionalInfoDiv > div > div:nth-child(1) > div.text.off > p')
    korean_abstract_text = korean_abstract.get_text(strip=True) if korean_abstract else ""

    multilingual_abstract = soup.select_one('#additionalInfoDiv > div > div:nth-child(2) > div.text.off > p')
    multilingual_abstract_text = multilingual_abstract.get_text(strip=True) if multilingual_abstract else ""

    year = soup.select_one('#thesisInfoDiv > div.infoDetail.on > div.infoDetailL > ul > li:nth-child(5) > div > p')
    year_text = year.get_text(strip=True) if year else ""

    summary_text = korean_abstract_text + "\n" + multilingual_abstract_text  # 국문과 영문의 위치가 논문마다 달라서 합침

    return {
        '저자': authors_text,
        '제목': title_kor,
        '키워드': keywords_text,
        '발행연도': year_text,
        '요약': summary_text.strip(), 
        '링크': url
    }

# 데이터를 CSV 파일에 저장하는 함수
def save_to_csv(data, csv_path):
    df = pd.DataFrame(data)
    if os.path.isfile(csv_path):
        df.to_csv(csv_path, mode='a', header=False, index=False, encoding='utf-8-sig')
    else:
        df.to_csv(csv_path, mode='w', header=True, index=False, encoding='utf-8-sig')

# 폴더를 생성하는 함수
def make_folder(folder_name):
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

In [9]:
if __name__ == "__main__":
    driver_path = 'chromedriver-win64/chromedriver.exe' # 압축 해제한 폴더
    chrome_service = Service(driver_path)

    # 크롬 옵션 설정
    chrome_options = webdriver.ChromeOptions()
    # headless 모드 설정(show_browser가 False일 때만 headless 적용)
    if not show_browser:
        chrome_options.add_argument("headless")
    
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

    user_name = getpass.getuser()
    folder_root = load_folder
    path = folder_root + now + '/'
    make_folder(path)

    csv_path = path + crawling_data_file_name
    all_paper_details = []

    start_page = 1
    end_page = page_count

    for page in range(start_page, end_page + 1):
        iStartCount = (page - 1) * 10  # 페이지당 10개씩
        # pageNumber, iStartCount 값에 맞춰 URL 생성
        url = URL_TEMPLATE.format(iStartCount=iStartCount)

        paper_links = get_paper_links(driver, url)

        # 2) 각 페이지별로 추출한 링크에 대해서 상세 정보 수집
        for link in paper_links:
            paper_details = get_paper_details(driver, link)
            all_paper_details.append(paper_details)

    # 3) 모든 페이지에 대해 수집한 결과를 CSV로 저장
    save_to_csv(all_paper_details, csv_path)

    driver.quit()

# 실행하는 동안 컴퓨터가 절전모드에 들어가면 안됨